**References:** https://towardsdatascience.com/depth-estimation-on-camera-images-using-densenets-ac454caa893



**Install required modules**

In [1]:
!pip install keras pillow matplotlib scikit-learn scikit-image opencv-python pydot GraphViz

In [2]:
!git clone https://github.com/priya-dwivedi/Deep-Learning.git

fatal: destination path 'Deep-Learning' already exists and is not an empty directory.


In [3]:
%cd Deep-Learning/depth_estimation

/content/Deep-Learning/depth_estimation


**Import modules**

In [4]:
from IPython.display import Image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import PIL
import os
import os.path
from zipfile import ZipFile
import os
import glob
import time
import argparse

# Kerasa / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from loss import depth_loss_function
from utils import predict, load_images, display_images, evaluate
from matplotlib import pyplot as plt

import numpy as np
from data import extract_zip
from io import BytesIO

#Import time module to time the cpu processing time for code execution
import time
# Import shutil to copy files
import shutil

Using TensorFlow backend.


**Download nyu v2 dataset**

In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu_data.zip

--2019-12-26 19:16:29--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu_data.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.106.131
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.106.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4399138715 (4.1G) [application/zip]
Saving to: ‘nyu_data.zip.2’

nyu_data.zip.2       22%[===>                ] 945.58M  1.82MB/s    eta 9m 38s 

**Download pre-trained weights from paper**

In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5

**Extract downloaded nyu v2 dataset data**

In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu_test.zip

**Perform PCA on test data to create new datasets**

In [0]:
# Load test data
print('Loading test data...', end='')

data = extract_zip('nyu_test.zip')

rgb = np.load(BytesIO(data['eigen_test_rgb.npy']))
depth = np.load(BytesIO(data['eigen_test_depth.npy']))
crop = np.load(BytesIO(data['eigen_test_crop.npy']))

In [0]:
type(rgb)
#rgbmat.shape = (rgb.size[1], rgb.size[0])
plt.figure(figsize=(9, 6))
plt.imshow(rgb[0])
plt.show()

In [0]:
# Evaluate modle on baseline (unmodified images):
!python evaluate.py --model ./nyu.h5

In [0]:
def to_rgb1(im):
    # I think this will be slow
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = im
    ret[:, :, 1] = im
    ret[:, :, 2] = im
    return ret

In [0]:
# Create list for each transformed image, so that the list can be transformed into a numpy array later
transformed_images_list = []

# Idea: 
# 0. Run evaluation on rgb file (zip) as baseline with unmodified data
# 0.5 store original rgb file so that original data can be read for every pca
# 1. Extract rgb file from zip and load data
# 2. Perform PCA with specified number of components for the data
# 3. Store modified data as rgb file
# 4. Zip rgb file with other two files for a new "nyu_test.zip" for the evaluation.py
# 5. Run evaluation.py for performance metrics
# 6. Go back to 2. and read the original rgb data before applying PCA, then evaluate on new data again


for components in [60,120,240,360,480]: #30
  print("Number of components:",str(components))
  # Iterate over all rgb images and replace 
  print("Modifying images...")
  for image_id in range(rgb.shape[0]):
    #print("Image:",image_id)
    # Read current image
    rgb_img = PIL.Image.fromarray(rgb[image_id])

    #print(rgb_imgmat.shape)

    rgb_imgmat = np.array(list(rgb_img.getdata(band=0)), float)
    

    # Reshape according to orginal image dimensions
    rgb_imgmat.shape = (rgb_img.size[1], rgb_img.size[0])
    
    number_of_singular_values = components

    # Singular value decomposition of image
    U, D, V = np.linalg.svd(rgb_imgmat)

    number_of_singular_values = components

    if(number_of_singular_values < rgb_imgmat.shape[0]):
      #start = time.process_time()
      reconstimg = np.matrix(U[:, :number_of_singular_values]) * np.diag(D[:number_of_singular_values]) * np.matrix(V[:number_of_singular_values, :])
      plt.imshow(reconstimg, cmap='gray') #
      plt.title("Reconstruction with "+ str(number_of_singular_values))

      transformed_images_list.append(to_rgb1(reconstimg))

  transformed_rgb = np.array(transformed_images_list)
  #transformed_rgb = np.expand_dims(transformed_rgb, 3)
  print(transformed_rgb.shape)

  #######################################
  # Evaluate after modification
  #######################################

  # Custom object needed for inference and training
  custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': depth_loss_function}

  # Load model into GPU / CPU
  print('Loading model...')
  model = load_model('nyu.h5', custom_objects=custom_objects, compile=False)


  # Load test data
  print('Loading test data...', end='')

  rgb = transformed_rgb

  print('Test data loaded.\n')


  start = time.time()
  print('Testing...')

  e = evaluate(model, rgb, depth, crop, batch_size=6)

  print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
  print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))

  end = time.time()
  print('\nTest time', end-start, 's')


#Number of components: 30
#Modifying images...
#(654, 480, 640, 3)
#Loading model...
#Loading test data...Test data loaded.
#
#Testing...
#        a1,         a2,         a3,        rel,        rms,     log_10
#    0.2438,     0.4956,     0.7263,     0.3698,     1.4703,     0.2147
#        a1,         a2,         a3,        rel,        rms,     log_10
#    0.2438,     0.4956,     0.7263,     0.3698,     1.4703,     0.2147
#
#Test time 173.71499300003052 s

In [0]:
transformed_rgb = np.array(transformed_images_list[0])
new_img = to_rgb1(transformed_rgb)
print(new_img.shape)

In [0]:
plt.figure(figsize=(9, 6))
plt.imshow(new_img)
plt.show()

**Evaluate**

**Visualize**

In [0]:
!python test.py --model ./nyu.h5

In [0]:
Image(filename="results.png") 

In [0]:
%cd Deep-Learning/depth_estimation
!dir

In [0]:
!dir my_examples

In [0]:
plt.style.use('classic')
img = Image.open('my_examples/test_IMG.jpg')
img = np.array(img)
img.shape

In [0]:
plt.imshow(img)
plt.show()

In [0]:
plt.style.use('classic')
img = Image.open('my_examples/IMG_0186.jpg')
# convert to numpy array
imgmat = np.array(list(img.getdata(band=0)), float)
print(imgmat.shape)
# Reshape according to orginal image dimensions
imgmat.shape = (img.size[1], img.size[0])

plt.figure(figsize=(9, 6))
plt.imshow(img)
plt.show()

**Singular value decomposition**

In [0]:
# Create folder for results
!mkdir "result_folder"

In [0]:
U, D, V = np.linalg.svd(img) #imgmat

In [0]:
import os, shutil
folder = 'my_examples/'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [0]:
#number_of_singular_values = 40
for number_of_singular_values in [2,4,8,32,64,125,250,561]:
  start = time.process_time()
  reconstimg = np.matrix(U[:, :number_of_singular_values]) * np.diag(D[:number_of_singular_values]) * np.matrix(V[:number_of_singular_values, :])
  plt.imshow(reconstimg) #, cmap='gray'
  plt.title("Reconstruction with "+ str(number_of_singular_values))
  plt.savefig('my_examples/test_IMG.jpg')
  plt.show()
  !python test.py --model ./nyu.h5
  print("Time for execution: ",time.process_time() - start)
  # Copy result file to designated folder
  shutil.copy2('results.png', 'result_folder/'+'result_with_'+str(number_of_singular_values)+'_components.png')


**Run depth estimation**

In [0]:
!python test.py --model ./nyu.h5

In [0]:
from IPython.display import Image

Image(filename="results.png") 